In [10]:
import threading, time

import time
#import util
import json
#import argparse
import sys
#import yaml
#from cloud_tools import Subscriber
import boto3
from hashlib import sha256, shake_256
import hashlib, json, time, enum 

import AWSIoTPythonSDK

import botocore
from shapely import Point
from shapely.geometry import Polygon

from AWSIoTPythonSDK import MQTTLib
from hashlib import sha256, shake_256
import hashlib, json, time, enum 


In [ ]:

def create_hash_dirs_underDev(records):
    geoindex_dir = results_dir.joinpath('geoindex')
    geoindex_dir.mkdir(exist_ok=True, parents=True)
    hashes = []
    for record in records:
        taxiCount = record.get('taxi_id', {})
        if taxiCount:
            latitude = taxiCount.get('latitude')
            longitude = taxiCount.get('longitude')
            if latitude and longitude:
                ## TODO: use pygeohash.encode() to assign geohashes to the records and complete the hashes list
                encoded_lat_long = pygeohash.encode(latitude,longitude)
                hashes.append(encoded_lat_long)
                record['geohash'] = encoded_lat_long
    hashes.sort()
    
    three_letter = sorted(list(set([entry[:3] for entry in hashes])))
    hash_index = {value: [] for value in three_letter}
    
    for record in records:
        geohash = record.get('geohash')
        if geohash:
            hash_index[geohash[:3]].append(record)
            
    for key, values in hash_index.items():
        output_dir = geoindex_dir.joinpath(str(key[:1])).joinpath(str(key[:2]))
        output_dir.mkdir(exist_ok=True, parents=True)
        output_path = output_dir.joinpath('{}.jsonl.gz'.format(key))
        with gzip.open(output_path, 'w') as f:
            json_output = '\n'.join([json.dumps(value) for value in values])
            f.write(json_output.encode('utf-8'))
            
    return list(set(hashes))


def taxi_searchDev(lat, lon, hashes_to_check, distance_in_km):
    ## TODO: Create simple search to return nearest airport
    distance_in_m = distance_in_km*1000
    cord_hash = pygeohash.encode(lat, lon)
    distance_list = []
    counter = 0
    for hash_ in hashes_to_check:
        distance_list.append(pygeohash.geohash_approximate_distance(cord_hash,hash_))
        if pygeohash.geohash_approximate_distance(cord_hash,hash_) == min(distance_list):
            closest_hash = hash_
            closest_distance = pygeohash.geohash_approximate_distance(cord_hash,hash_)
    if min(distance_list) <= distance_in_m:
        for record in records:
            if closest_hash == record.get('geohash'):
                print('The closest taxi is', record.get('taxi_id').get('taxi_type'), 'at a distance of', closest_distance/1000, 'kilometers.')
                counter += 1
            if counter >=1:
                break
    else:
        print("No taxi found in the distance specified.")

#taxi_search(37.958182, -122.040756, hash_list, 500)
#hash_list = create_hash_dirs(records)

In [11]:

taxyType = enum.Enum('taxyType',
                  ['Utility', 'Deluxe', 'Luxury', 'RideShare'])


def make_hashes(password):
	return shake_256(str.encode(password)).hexdigest(77)

aPieceofPriData = 'Hello'
make_hashes(aPieceofPriData)


def make_hashes_len(password, lenOfhash=int):
	return shake_256(str.encode(password)).hexdigest(lenOfhash)

aPieceofPriData = 'Hello'
make_hashes_len(aPieceofPriData, 5)

def check_hashes(password, hashed_text):
    if make_hashes(password) == hashed_text:
        return hashed_text
    
    return False


In [12]:
aPieceofPriData = 'Hello'


class Blockchain():
    def __init__(self):
        self.chain = [self.create_genesis_block()]
        self.temp_data = []
    
    def create_genesis_block(self):
        return {"index":0,
                'timestamp': time.time(),
                'data': None,
                'prev_hash': None} 
        
    def new_taxi(self, taxi_id, taxi_type):
        taxi = {'taxi_id':taxi_id,
                'taxi_type': taxi_type}
                
        self.temp_data.append(taxi)
        
    def new_User(self, UserId, number):
        user = {'UserId':UserId,
                'UserNumber': number}
                
        self.temp_data.append(user)
        
    def add_block(self, proof_of_work):
        block = {'index':len(self.chain) + 1,
                 'timestamp': time.time(),
                 'data': self.temp_data,
                 'prev_hash': self.hash_code(self.chain[-1]),
        }
        self.temp_data = []
        self.chain.append(block)
        return block
    
    def hash_code (self, block):
        str_obj = json.dumps(block, sort_keys=True)
        str = str_obj.encode()
        hash_number = hashlib.sha256(str)
        hexadeimal_hash = hash_number.hexdigest()
        return hexadeimal_hash
    
block_chain = Blockchain()  

In [13]:
name = 'james'
email = 'email@mail.com'
number = 5465456456

class newUser :
      
    def __init__(self, name, email, number):                 
        self.name = name
        self.email = email 
        self.number = number
        #self.address = eth  address
        self.customerId = make_hashes_len(name, 5)
        #self.customerId = hex(hash(name) - hash(time.time()**2))
        
        self.BorrowerDetails = {
            
            'Customer Name' : self.name,
            'Customer Email' : self.email,
            'Customer number': self.number,
            'Customer Id': self.customerId,
            
           #  'Customer BC address': self.address
        } 

newUsertest = newUser(name, email, number)


In [14]:

aPieceofPriData = 'Hello'

u000 = block_chain.new_User(newUsertest.customerId, newUsertest.number)


d001 = block_chain.new_taxi(make_hashes_len(aPieceofPriData, 7), str(taxyType.RideShare))
d002 = block_chain.new_taxi(make_hashes_len('aPieceofPriData', 7), str(taxyType.Deluxe))
d003 = block_chain.new_taxi(make_hashes_len('fadsffa', 7), str(taxyType.Luxury))

block_chain.add_block(151199)


{'index': 2,
 'timestamp': 1674607536.3206074,
 'data': [{'UserId': '16a37ff6a2', 'UserNumber': 5465456456},
  {'taxi_id': '555796c90bfb8f', 'taxi_type': 'taxyType.RideShare'},
  {'taxi_id': '9ea90bc1239c00', 'taxi_type': 'taxyType.Deluxe'},
  {'taxi_id': '19fc1f8043eb64', 'taxi_type': 'taxyType.Luxury'}],
 'prev_hash': '84288620b74bc8a2e69a52934bef36d6e67f7e91287b01e8a97724a0ce684725'}

In [15]:

PriData = 'PrivateKey'


d006 = block_chain.new_taxi(make_hashes_len(PriData, 7), str(taxyType.RideShare))
d007 = block_chain.new_taxi(make_hashes_len('PriData', 7), str(taxyType.Utility))
d008 = block_chain.new_taxi(make_hashes_len('HelloWorld', 7), str(taxyType.Deluxe))

block_chain.add_block(651199)


{'index': 3,
 'timestamp': 1674607536.9946263,
 'data': [{'taxi_id': 'd0020ea7254796', 'taxi_type': 'taxyType.RideShare'},
  {'taxi_id': '21a6303c62ba34', 'taxi_type': 'taxyType.Utility'},
  {'taxi_id': '869337c17f5d9a', 'taxi_type': 'taxyType.Deluxe'}],
 'prev_hash': 'd355759d17c2b90f958e34910f26d6d9330d4c9ca34419d4e2337ca7d9f2be0b'}

In [16]:

ret = block_chain.chain[1].get('data')

display(
ret[1]['taxi_id'],
ret[1]['taxi_type']
)



'555796c90bfb8f'

'taxyType.RideShare'

In [8]:

# aws credentials below

aws_access_key_id = 'aws_access_key_id'
aws_secret_access_key = 'aws_secret_access_key'

region = "US West)"
endpoint = 'iot.us-west-1.amazonaws.com'

arn = ':aws:iot:us'

root_ca = "./AmazonRoot.pem"

key = "private.pem.key"

cert = "certificate.pem.crt"

client_id = "TaxY"

iot = boto3.client('iot', region_name='us-west-2', aws_access_key_id=aws_access_key_id,
         aws_secret_access_key= aws_secret_access_key)


dynamodb = boto3.resource('dynamodb', region_name='us-west-2', aws_access_key_id=aws_access_key_id,
         aws_secret_access_key= aws_secret_access_key)

table = dynamodb.Table('taxis')

# Define the service area using a lat/long bounding box
service_area = Polygon([(37.788022, -122.399797), (37.788022, -122.391777), (37.794546, -122.391777), (37.794546, -122.399797)])

# Check if a given location (taxi's location) is within the service area
taxi_location = (37.790519, -122.397530)
if service_area.contains(Point(taxi_location)):
    print("Taxi is within the service area.")
else:
    print("Taxi is outside the service area.")
    
    


Taxi is within the service area.


In [9]:
class Subscriber(threading.Thread):
    """A threaded Subscriber object"""

    def __init__(self, endpoint, root_ca, key, cert, client_id=''):
        threading.Thread.__init__(self)
        self.endpoint = endpoint
        self.client_id = client_id
        self.root_ca = root_ca
        self.key = key
        self.cert = cert
        self._client = None
        self.finish = False
        self.daemon = True
        self.connected = False

    def connect(self):
        self._client = MQTTLib.AWSIoTMQTTClient(self.client_id)
        self._client.configureEndpoint(self.endpoint, 8883)
        self._client.configureCredentials(self.root_ca, self.key, self.cert)
        self._client.configureOfflinePublishQueueing(-1)  # Infinite offline Publish queueing
        self._client.configureConnectDisconnectTimeout(10)  # 10 sec
        self._client.configureMQTTOperationTimeout(5)  # 5 sec
        self.connected = self._client.connect()

    def subscribe(self, topic, callback, qos=1):
        if not self.connected:
            self.connect()
        self._client.subscribe(topic, qos, callback)

    def run(self):
        while not self.finish:
            time.sleep(0.001)

In [ ]:
class Taxi:
    """Encapsulates an Amazon DynamoDB table of taxi data."""
    def __init__(self, dyn_resource):
        """
        :param dyn_resource: A Boto3 DynamoDB resource.
        """
        self.dyn_resource = dyn_resource
        self.table = None
    
    def create_table(self, table_name):
        """
        Creates an Amazon DynamoDB table that can be used to store taxi data.
        The table uses the release year of the taxi as the partition key and the
        title as the sort key.
        4485
        AWS SDK Code Examples Code Library
        Actions and scenarios
        :param table_name: The name of the table to create.
        :return: The newly created table.
        """
        try:
            self.table = self.dyn_resource.create_table(
            TableName=table_name,
            KeySchema=[
            {'AttributeName': 'taxi_id', 'KeyType': 'HASH'}, # Partition key
            {'AttributeName': 'taxi_type', 'KeyType': 'RANGE'} # Sort key
    ],
            AttributeDefinitions=[
            {'AttributeName': 'taxi_id', 'AttributeType': 'N'},
            {'AttributeName': 'taxi_type', 'AttributeType': 'S'}
            ],
            ProvisionedThroughput={'ReadCapacityUnits': 10, 'WriteCapacityUnits':
            10})
            self.table.wait_until_exists()
        except botocore.exceptions.ClientError as err:
            botocore.logger.error("Couldn't create table %s. Here's why: %s: %s", table_name,err.response['Error']['Code'], err.response['Error']['Message'])
            
            raise
        else:
            
            return self.table


In [ ]:
taxi_table = Taxi(dynamodb)
taxi_table.create_table('taxiTable')
